In [1]:
import pandas as pd
# pd.set_option("display.max_rows", None)
def cleanffball(year): # add name to inputs if wanting to create csv
    import pandas as pd
    url = f'https://www.pro-football-reference.com/years/{year}/fantasy.htm'
    df = pd.read_html(url,header=1)[0]
    df = df.drop(['Rk','Tm','VBD','FantPt','DKPt','FDPt'],axis=1)
    df = df[df['FantPos'].notna()]
    df = df[df['FantPos'] != 'FantPos']
    df['Y/A'] = df['Y/A'].fillna(0)
    df['Y/R'] = df['Y/R'].fillna(0)
    df['2PM'] = df['2PM'].fillna(0)
    df['2PP'] = df['2PP'].fillna(0)
    df = df.replace('\+','',regex=True).astype(object)
    df = df.replace('\*','',regex=True).astype(object)
    df['Player'] = df['Player'].str.rstrip()
    columns = df.columns[2:]
    for col in columns:
        df[col] = pd.to_numeric(df[col],errors='coerce')
    df['PPR_OvRank'] = df['PPR'].rank(method='first',ascending=False)
    df = df.drop(columns=['OvRank'],axis=1)
    df = df.dropna()
    df['Year'] = year
    df['PPG'] = df['PPR'] / df['G']
    df['PPG'] = df['PPG'].round(decimals = 3)
    # edits name for uniformity year over year
    df['Player'] = df.Player.str.replace('[^a-zA-Z]', '', regex = True)
    # creates unique player ID to make stats for different years easier to see
    playerID = []
    for index, row in df.iterrows():
        playerID.append(row['Player'] + str(row['Year']))
    df['ID'] = playerID
    return df
    # df.to_csv(f'{name}.csv',index=False)

In [2]:
def createDB(startYear, endYear):
    database = pd.DataFrame()
    years = []
    # iterates through years specified
    for year in range(startYear, endYear + 1):
        years.append(year)
    for year in years:
        database = database.append(cleanffball(year))
    return database

In [3]:
def cleanDB(database):
    cols = []
    for column in database.columns:
        if column == 'FantPos':
            cols.append('Pos')
        elif column == 'Yds':
            cols.append('PassYds')
        elif column == 'Yds.1':
            cols.append('RushYds')
        elif column == 'Yds.2':
            cols.append('RecYds')
        elif column == 'Att':
            cols.append('PassAtt')
        elif column == 'Att.1':
            cols.append('RushAtt')
        elif column == 'TD':
            cols.append('PassTD')
        elif column == 'TD.1':
            cols.append('RushTD')
        elif column == 'TD.2':
            cols.append('RecTD')
        elif column == 'TD.3':
            cols.append('TotTD')
        else:
            cols.append(column)

    database.columns = cols
    
    listOfColumns = ['Cmp','PassAtt','PassYds','PassTD','Int','RushAtt','RushYds',
                     'Y/A','RushTD','Tgt','Rec','RecYds','Y/R','RecTD','Fmb',
                     'FL','TotTD','2PM','2PP','PPR','PosRank','PPR_OvRank','Year',
                     'PPG']
    
    # Creates columns for stats per game
    for column in listOfColumns:
        if column == 'Y/A':
            newColumn = 'RushYds/Att'
            database[newColumn] = database[column]
        elif column == 'Y/R':
            newColumn = 'Yds/Rec'
            database[newColumn] = database[column]
        elif column == 'PPR':
            newColumn = 'FPts'
            database[newColumn] = database[column]
        elif column == 'PosRank':
            newColumn = 'PosRk'
            database[newColumn] = database[column]
        elif column == 'PPR_OvRank':
            newColumn = 'OvRank'
            database[newColumn] = database[column]
        elif column == 'Year':
            newColumn = 'Yr'
            database[newColumn] = database[column]
        elif column == 'PPG':
            newColumn = 'PPR/G'
            database[newColumn] = database[column]
        else:
            newColumn = column + '/G'
            database[newColumn] = (pd.to_numeric(database[column]) / pd.to_numeric(database['G']))
            database[newColumn] = database[newColumn].round(decimals = 3)
            
    database = database.drop(columns = listOfColumns)
            
    database = database.sort_values(by = ['Player','Yr'], ascending = [True,False])
    nextYearPPG = []
    lastPlayer = 'NaN'
    lastPlayerPPG = 'NaN'

    for index, row in database.iterrows():
            player = row['Player']
            if lastPlayer == player:
                nextYearPPG.append(lastPlayerPPG)
            else:
                nextYearPPG.append('NaN')
            lastPlayer = row['Player']
            lastPlayerPPG = row['PPR/G']
    
    database['Next_Yr_PPG'] = nextYearPPG
    database['Next_Yr_PPG'] = pd.to_numeric(database['Next_Yr_PPG'],errors='coerce')
    database = database.reset_index(drop = True)
    return database

In [4]:
db = cleanDB(createDB(2012, 2021))
db

,Player,Pos,Age,G,GS,ID,Cmp/G,PassAtt/G,PassYds/G,PassTD/G,...,FL/G,TotTD/G,2PM/G,2PP/G,FPts,PosRk,OvRank,Yr,PPR/G,Next_Yr_PPG
0,AJBrown,WR,24,13,13,AJBrown2021,0.0,0.154,0.0,0.0,...,0.000,0.385,0.000,0.0,180.9,32,85.0,2021,13.915,NaN
1,AJBrown,WR,23,14,12,AJBrown2020,0.0,0.000,0.0,0.0,...,0.071,0.857,0.000,0.0,247.5,9,38.0,2020,17.679,13.915
2,AJBrown,WR,22,16,11,AJBrown2019,0.0,0.000,0.0,0.0,...,0.000,0.562,0.000,0.0,217.1,9,62.0,2019,13.569,17.679
3,AJDerby,TE,27,4,0,AJDerby2018,0.0,0.000,0.0,0.0,...,0.000,0.250,0.000,0.0,13.8,76,400.0,2018,3.450,NaN
4,AJDerby,TE,26,11,1,AJDerby2017,0.0,0.000,0.0,0.0,...,0.091,0.182,0.000,0.0,55.4,41,257.0,2017,5.036,3.450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5462,ZayJones,WR,24,15,9,ZayJones2019,0.0,0.000,0.0,0.0,...,0.000,0.000,0.000,0.0,48.9,134,269.0,2019,3.260,2.288
5463,ZayJones,WR,23,16,15,ZayJones2018,0.0,0.062,0.0,0.0,...,0.000,0.438,0.062,0.0,165.2,34,93.0,2018,10.325,3.260
5464,ZayJones,WR,22,15,10,ZayJones2017,0.0,0.000,0.0,0.0,...,0.000,0.133,0.000,0.0,70.6,90,218.0,2017,4.707,10.325
5465,ZurlonTipton,RB,25,10,0,ZurlonTipton2015,0.0,0.000,0.0,0.0,...,0.000,0.000,0.000,0.0,12.7,124,419.0,2015,1.270,NaN


In [5]:
db.columns

Index(['Player', 'Pos', 'Age', 'G', 'GS', 'ID', 'Cmp/G', 'PassAtt/G',
       'PassYds/G', 'PassTD/G', 'Int/G', 'RushAtt/G', 'RushYds/G',
       'RushYds/Att', 'RushTD/G', 'Tgt/G', 'Rec/G', 'RecYds/G', 'Yds/Rec',
       'RecTD/G', 'Fmb/G', 'FL/G', 'TotTD/G', '2PM/G', '2PP/G', 'FPts',
       'PosRk', 'OvRank', 'Yr', 'PPR/G', 'Next_Yr_PPG'],
      dtype='object')

In [6]:
db.dtypes

Player          object
Pos             object
Age              int64
G                int64
GS               int64
ID              object
Cmp/G          float64
PassAtt/G      float64
PassYds/G      float64
PassTD/G       float64
Int/G          float64
RushAtt/G      float64
RushYds/G      float64
RushYds/Att    float64
RushTD/G       float64
Tgt/G          float64
Rec/G          float64
RecYds/G       float64
Yds/Rec        float64
RecTD/G        float64
Fmb/G          float64
FL/G           float64
TotTD/G        float64
2PM/G          float64
2PP/G          float64
FPts           float64
PosRk            int64
OvRank         float64
Yr               int64
PPR/G          float64
Next_Yr_PPG    float64
dtype: object